# LC gauge transformation of Glasma using gauge fields
Done following Meijian's Mathematica notebook.

### Extract the gauge fields
First, let's extract the Glasma gauge fields $A^x(x,y,\tau)$ and $A^y(x,y,\tau)$ as logarithms of the corresponding gauge links. We will use the same simulation parameters as in the `Light cone gauge transformation of the Glasma fields.ipynb` notebook.

#### Simulation parameters

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
N = 128   
tau_sim = 1     
DTS = 8     

# Glasma fields
su_group = 'su3'
Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

# TODO: Run more events
nevents = 1

#### Environment variables

In [2]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
import curraun.lattice as latt
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
# Number of colors
Nc = su.NC
# Dimension of algebra 
Dg = su.GROUP_ELEMENTS

Using Numba
Using SU(3)
Using double precision


### 1. Test the extraction of the gauge field

#### Simulation routine for evolving the Glasma fields
The gauge field is extracted as the logarithm of the gauge link 
$$\boxed{A_i(x,y,\tau)=\,\mathrm{ln} \,U_i(x,y,\tau)},$$
with $i=x,y$ and $g$ and $a$ absorbed in the numerical definition of the gauge field. Notice that the gauge field is not evaluated in the midpoint $x+a/2$ since this is already a $\mathcal{O}(a^2)$ correction. As extracted in the notebook [Extract the logatirhm of a SU(3) gauge link
](https://github.com/avramescudana/curraun/blob/wong/notebooks/Logarithm%20of%20a%20SU(3)%20gauge%20link.ipynb), the fastest and simplest way to extract the logarithm of a gauge link is using the anti-hermitian part 
$$\boxed{\mathrm{ln}\,U=\dfrac{U-U^\dagger}{2}}.$$ 
It is equivalent to performing a Taylor series of the logarithm,

In [3]:
def ln_ah(ui):
    buf0 = su.dagger(ui)
    buf1 = su.mul_s(buf0, -1)
    buf2 = su.add(ui, buf1)
    # buf3 = su.mul_s(buf2, -0.5j)
    buf3 = su.mul_s(buf2, 0.5)
    ai = np.array(buf3)
    return ai

Alternatively, extract the logarithm of a gauge link as a Taylor series.

In [4]:
def ln_taylor(ui):
    buf1 = su.mlog(ui)
    # buf2 = su.mul_s(buf1, -1.0j)
    buf2 = su.mul_s(buf1, 1.0)
    ai = np.array(buf2)
    return ai

In [5]:
import pickle
from tqdm import tqdm

# Simulation routine
def simulate(): 

    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Transverse gauge links
    ax = su.GROUP_TYPE(np.zeros((maxt//DTS, N*N, Dg)))
    ax_taylor = su.GROUP_TYPE(np.zeros((maxt//DTS, N*N, Dg)))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):            
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            # Copy gauge links and gauge fields from the memory of the GPU
            if t%DTS == 0:
                ui = s.u1.copy()
                ax[t//DTS] = np.apply_along_axis(ln_ah, 1, ui[:, 0, :])
                ax_taylor[t//DTS] = np.apply_along_axis(ln_taylor, 1, ui[:, 0, :])
                        
            pbar.update(1)

    if use_cuda:
        cuda.current_context().deallocations.clear()

    return ax, ax_taylor

#### Run the simulation 

In [6]:
# Supress various horribly long warnings
import warnings
warnings.filterwarnings('ignore')

In [7]:
ax_ah, ax_taylor = simulate()

100%|██████████| 512/512 [03:14<00:00,  2.63it/s]


In [8]:
ax_taylor_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 0, ax_taylor[:,:,:])
ax_ah_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 0, ax_ah[:,:,:])

ax_taylor_comp[1,1,:] - ax_ah_comp[1,1,:] 

array([-4.04308346e-16,  1.57188497e-05,  5.72589787e-05, -1.57188497e-05,
        2.83908176e-16, -3.07097789e-05, -5.72589787e-05,  3.07097789e-05,
       -6.20648342e-16])

- - -

### 2. Extract the gauge fields

In [9]:
def ln_ah(ui):
    buf0 = su.dagger(ui)
    buf1 = su.mul_s(buf0, -1)
    buf2 = su.add(ui, buf1)
    # buf3 = su.mul_s(buf2, -0.5j)
    buf3 = su.mul_s(buf2, 0.5)
    ai = np.array(buf3)
    return ai

In [10]:
import pickle
from tqdm import tqdm

# Simulation routine
def simulate(): 

    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Transverse gauge links
    ax = su.GROUP_TYPE(np.zeros((maxt//DTS, N*N, Dg)))
    ay = su.GROUP_TYPE(np.zeros((maxt//DTS, N*N, Dg)))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):            
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            # Copy gauge links and gauge fields from the memory of the GPU
            if t%DTS == 0:
                ui = s.u1.copy()
                ax[t//DTS] = np.apply_along_axis(ln_ah, 1, ui[:, 0, :])
                ay[t//DTS] = np.apply_along_axis(ln_ah, 1, ui[:, 1, :])
                        
            pbar.update(1)

    if use_cuda:
        cuda.current_context().deallocations.clear()

    return ax, ay

In [11]:
# Supress various horribly long warnings
import warnings
warnings.filterwarnings('ignore')

In [12]:
ax, ay = simulate()

100%|██████████| 512/512 [00:34<00:00, 14.66it/s]


In [13]:
ax_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 0, ax[:,:,:])
ay_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 0, ay[:,:,:])

### 3. Construct the gauge transformation

In the temporal gauge $A^\tau=0$ and at midrapidity $\eta=z=0$ the Glasma fields in LC coordinates are expressible as 
$$A^\pm_\mathrm{temp}(x^+,x^-,y,z=0)=\pm C A^x(\tau,x,y),\quad A^y_\mathrm{temp}(x^+,x^-,y,z=0)=A^y(\tau,x,y).$$ 
Here $C$ is a constant appearing in the definition of the LC coordinates $x^\pm = C(t\pm x)$. We are interested in expressing these fields in a LC lattice. 

Additionally, we work in the eikonal limit for which the $x^-$ dependence is not relevant. Thus, we choose $x^-=0$ and construct the LC gauge transformation operator in a triangular lattice. The gauge transformation is given by 
$$\mathcal{U}^\dagger_\mathrm{LC}(x^+,y,z=0)=\prod\limits_k \mathcal{U}^\dagger_\mathrm{LC}(x^+,x^-_k,y,z=0),$$ 
where each infinitesimal piece is 
$$\mathcal{U}^\dagger_\mathrm{LC}(x^+,x^-_k,y,z=0)=\mathrm{exp}\Bigg\{\dfrac{\mathrm{i}g}{2C^2}\delta x^- A^+_\mathrm{temp}(x^+,x^-_k,y,z=0)\Bigg\}.$$ 

If we work with a LC lattice of spacing $\delta x^-=\sqrt{2}a$ suitable for a LC convention $C=1/\sqrt{2}$, the infinitesimal LC gauge trasformation becomes 
$$\mathcal{U}^\dagger_\mathrm{LC}(x^+,x^-_k,y,z=0)=\mathrm{exp}\Big\{\mathrm{i}g a A^x(x^+,x^-_k,y,z=0)\Big\}.$$ 

- - -

Let us closely follow Carlos' code for constructing the LC gauge transformation from [LC Coordinate Lattice](https://github.com/avramescudana/curraun/blob/jets/notebooks/LC%20Coordinate%20Lattice.ipynb).

In [14]:
t_steps = np.shape(ax)[0]
N = np.int(np.sqrt(np.shape(ax)[1]))
Dg = np.shape(ax)[2]

In [15]:
ax_r = ax.reshape((t_steps, N, N, Dg))
ay_r = ay.reshape((t_steps, N, N, Dg))

#### 3.1 We write the relevant magnitude in LC coordinates

In [16]:
# We define a function that, given a set of gauge fields, returns the values over the x^+ axis
# Notice that we do not carry the constant C because in the end it will "cancel" out with an appropiate choice of LC lattice spacing

def get_xplus_axis_fields (a):
    aplus = su.GROUP_TYPE(np.zeros((t_steps, N, Dg)))
    for t in range (t_steps):
        aplus[t, :, :] = a[t,t,:,:]
    return aplus

In [17]:
aplus_r = get_xplus_axis_fields(ax_r)

In [18]:
aplus_r.shape

(64, 128, 9)

#### 3.2 We gauge transform the relevant quantity to LC gauge

In [19]:
def exp_gauge_field(a):
    # return su.mexp(a)
    exp = np.apply_along_axis(su.mexp, 1, a)
    return exp

In [20]:
# We construct a function that gives the gauge operator at a given point

def gauge_operator(a, xplus, xminus):
    v = su.GROUP_TYPE(np.array([su.unit() for i in range(N)]))
    for t in range(xplus+xminus):
        buffer1 = v
        # Here I rexpondentiate the logarithm of a gauge link. It is quite redundant
        #TODO: Use here directly the gauge link? 
        aux = exp_gauge_field(a[t, 2*xplus-t, :, :])
        buffer2 = np.apply_along_axis(su.dagger, 1, aux)
        v = np.array([su.mul(buffer1[i, :], buffer2[i, :]) for i in range(N)])
    return v

In [21]:
# We construct an array with the gauge operator over the x^+ axis

v_LC = su.GROUP_TYPE(np.zeros((t_steps, N, Dg)))

for t in range (t_steps):
    v_LC[t,:,:] = gauge_operator(ax_r, t, 0)

In [23]:
v_LC.shape

(64, 128, 9)

#### 3.2 We apply the gauge transformation
This is the part that is different from Carlos' code from [LC Coordinate Lattice](https://github.com/avramescudana/curraun/blob/jets/notebooks/LC%20Coordinate%20Lattice.ipynb).

The gauge transformation is done at the level of the gauge fields, namely 
$$A_\mu^\mathrm{LC}(x_\mathrm{LC}^\mathrm{eik})=\mathcal{U}_\mathrm{LC}(x_\mathrm{LC}^\mathrm{eik}) A_\mu^\mathrm{temp}(x_\mathrm{LC}^\mathrm{eik}) \mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}^\mathrm{eik})+\dfrac{\mathrm{i}}{g}\mathcal{U}_\mathrm{LC}(x_\mathrm{LC}^\mathrm{eik})\partial_\mu \mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}^\mathrm{eik}),$$ 
where $x_\mathrm{LC}^\mathrm{eik}\equiv (x^+, y, z=0)$. Notice that there is no $x^-$ dependence since we work in the eikonal limit.

In her approach, Meijian applies the partial derivative on the gauge transformation as 
$$\partial_\mu \mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}^\mathrm{eik})=\dfrac{\mathrm{i}g}{2C^2}\delta x^- \sum\limits_k \partial_\mu A^+_\mathrm{temp}(x^+,x^-_k,y,z=0)\,\,\mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}^\mathrm{eik}).$$

Further, the partial derivative of the gauge field may be extracted as
$$\partial_\mu A_\mu^\mathrm{LC}(x_\mathrm{LC})\equiv \dfrac{1}{2\delta x^\mu}\Big[A_\mu^\mathrm{LC}(x_\mathrm{LC}+\delta x^\mu\hat{e}_\mu)-A_\mu^\mathrm{LC}(x_\mathrm{LC}-\delta x^\mu\hat{e}_\mu)\Big]$$
which represent the symmetric definition of the partial derivative on the lattice. Here $\delta x^\mu$ is the lattice spacing along the direction $\hat{e}_\mu$ and $x_\mathrm{LC}\pm\delta x^\mu\hat{e}_\mu$ is the four vector where the component $\mu$ is displaced by $\pm\delta x^\mu$. In our LC lattice construction, $C=1/\sqrt{2}$ and $\delta x^\pm = \sqrt{2}a$.

- - -
Alternatively, one may directly extract the partial derivative of the gauge transformation operator in the LC lattice as
$$\partial_\mu \mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}^\mathrm{eik})\equiv \dfrac{1}{2\delta x^\mu}\Big[\mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}^\mathrm{eik}+\delta x^\mu\hat{e}_\mu)^\mathrm{eik}-\mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}-\delta x^\mu\hat{e}_\mu)\Big]$$
but this would yield incorrect results for the $\partial_-$ derivative since we dropped the $x^-$ dependence in $U^\dagger(x_\mathrm{LC}^\mathrm{eik})$. The $x^-$ derivative will be $\partial_- \mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}^\mathrm{eik})=0$, but the LC time $x^+$ derivative will have a finite contribution as
$$\partial_+ \mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}^\mathrm{eik})\equiv \dfrac{1}{2\delta x^+}\Big[\mathcal{U}_\mathrm{LC}^\dagger(x^++\delta x^+, y, z=0)-\mathcal{U}_\mathrm{LC}^\dagger(x^+-\delta x^+, y, z=0)\Big]$$
and similarly for spatial $y$ derivative
$$\partial_y \mathcal{U}_\mathrm{LC}^\dagger(x_\mathrm{LC}^\mathrm{eik})\equiv \dfrac{1}{2\delta y}\Big[\mathcal{U}_\mathrm{LC}^\dagger(x^+, y+\delta y, z=0)-\mathcal{U}_\mathrm{LC}^\dagger(x^+, y-\delta y, z=0)\Big]$$

- - -
Gauge transformation of $A^+_\mathrm{temp}$, which should yield $A^+_\mathrm{LC}=0$.

In [32]:
aplus_LC = su.GROUP_TYPE(np.zeros(np.shape(v_LC)))

In [33]:
for i in range(v_LC.shape[0]):
    for j in range(v_LC.shape[1]):
        aplus_LC[i,j,:] = latt.act(aplus_r[i,j,:], v_LC[i,j,:])

In [35]:
aplus_r[0, 0, :]

array([ 0.        +0.02572523j,  0.06554996+0.0065007j ,
        0.04576416+0.03783409j, -0.06554996+0.0065007j ,
        0.        +0.10690635j, -0.03971413-0.01257774j,
       -0.04576416+0.03783409j,  0.03971413-0.01257774j,
        0.        -0.1325716j ])

In [36]:
aplus_LC[0, 0, :]

array([ 0.00852662-9.00091144e-20j, -0.00143115-9.62093310e-03j,
       -0.00152094+5.97237457e-03j, -0.00143115+9.62093310e-03j,
        0.01750344-1.05727315e-20j,  0.0035686 +1.16325066e-03j,
       -0.00152094-5.97237457e-03j,  0.0035686 -1.16325066e-03j,
        0.02283642+1.18992949e-19j])

In [38]:
su.mexp(aplus_LC[0, 0, :])

((1.0086301002076181-9.087632922040728e-20j),
 (-0.0014491686502802032-0.009735508356578446j),
 (-0.0015419368293170558+0.006048673624108561j),
 (-0.0014491686502802032+0.009735508356578446j),
 (1.0177126206572829-1.0974219441042696e-20j),
 (0.0036133043488473064+0.0011752012276957653j),
 (-0.0015419368293170558-0.006048673624108561j),
 (0.0036133043488473064-0.0011752012276957653j),
 (1.0231256202296646+1.2171592333920434e-19j))

- - -
### Output Glasma fields to files
Write $A_x$, $A_y$ and $A_\eta$ to `*.txt` files. Each time $\tau$ slice folder contains a file whose name is formated as `Ax_x_4_y_6.txt` with all the color components of the gauge field at that lattice location.

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box           

N = 8  
L = 0.4 
tau_sim = 0.8 

# N = 32  
# L = 1.6  

N = 64  
L = 1.6  
# Such that Qs a << 1

DTS = 4   

# Glasma fields
su_group = 'su3'
Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

# TODO: Run more events
nevents = 1

In [2]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
import curraun.lattice as latt
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
# Number of colors
Nc = su.NC
# Dimension of algebra 
Dg = su.GROUP_ELEMENTS

Using Numba
Using SU(3)
Using double precision


In [3]:
def ln_ah(ui):
    buf0 = su.dagger(ui)
    buf1 = su.mul_s(buf0, -1)
    buf2 = su.add(ui, buf1)
    # buf3 = su.mul_s(buf2, -0.5j)
    buf3 = su.mul_s(buf2, 0.5)
    ai = np.array(buf3)
    return ai

In [4]:
import pickle
from tqdm import tqdm

# Simulation routine
def simulate(): 

    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Transverse gauge links
    # ax = su.GROUP_TYPE(np.zeros((maxt, N*N, Dg)))
    # ay = su.GROUP_TYPE(np.zeros((maxt, N*N, Dg)))
    # aeta = su.GROUP_TYPE(np.zeros((maxt, N*N, Dg)))
    a = su.GROUP_TYPE(np.zeros((maxt//DTS, 3, N*N, Dg)))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):            
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            # Copy gauge links and gauge fields from the memory of the GPU

            if t%DTS == 0:
                ui = s.u1.copy()
                # ax[t] = np.apply_along_axis(ln_ah, 1, ui[:, 0, :])
                # ax[t] = np.apply_along_axis(ln_ah, 1, ui[:, 1, :])
                # aeta[t] = s.aeta1.copy()

                lattice_dimension = g * a
                #TODO: The fields are outputed in lattice units, should I convert?
                physical_units = E0 ** 2
                #TODO: Should I output the gauge links in physical units?

                a[t//DTS, 0, :, :] = np.apply_along_axis(ln_ah, 1, ui[:, 0, :])
                a[t//DTS, 1, :, :] = np.apply_along_axis(ln_ah, 1, ui[:, 1, :])
                a[t//DTS, 2, :, :] = s.aeta1.copy()
                        
            pbar.update(1)

    if use_cuda:
        cuda.current_context().deallocations.clear()

    # return ax, ay, aeta
    return a

In [5]:
# Supress various horribly long warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# ax, ay, aeta = simulate()
a = simulate()

100%|██████████| 128/128 [00:13<00:00,  9.38it/s]


In [7]:
# ax_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 2, ax)
# ay_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 2, ay)
# aeta_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 2, aeta)

a_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 3, a)

In [8]:
comps = ["ax", "ay", "aeta"]
# comps = ["ax"]

for mu in range(len(comps)):
    folder_path = "results/glasma_" + str(N) + "x" + str(N) + "_latt_" + comps[mu] + "_comp/"
    amu = a_comp[:, mu, :, :]

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Output all color components in the xy plane, each time slice in a different file
    for it in range(amu.shape[0]):
        output_file = open(folder_path + "tau_" + str(it+1) + ".txt", "a")
        for ix in range(amu.shape[1]):
            for ic in range(amu.shape[2]):
                if ic==(amu.shape[2]-1):
                    output_file.write(str(amu[it, ix, ic]) + "\n")
                else:
                    output_file.write(str(amu[it, ix, ic]) + " ")
    output_file.close()

In [ ]:
# Older test version, a single component
folder_path = "ax_glasma/"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Output all color components in the xy plane, each time slice in a different file
for it in range(ax_comp.shape[0]):
    output_file = open(folder_path + "tau_" + str(it) + ".txt", "a")
    for ix in range(ax_comp.shape[1]):
        for ic in range(Dg-1):
            if ic==(Dg-2):
                output_file.write(str(ax_comp[it, ix, ic]) + "\n")
            else:
                output_file.write(str(ax_comp[it, ix, ic]) + " ")
output_file.close()